## Import Packages

In [8]:
import csv
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import tree

## Extract Data

In [11]:
def findCommunities(filename):
    communities = []
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    xPositive=[]
    yPositive=[]
    xNegative=[]
    yNegative =[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[5:len(i)-1]
                while '?' in data:
                    data[data.index('?')] = 0
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                #x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    xPositive.append(vect.astype(np.float))
                    yPositive.append(1)
                else:
                    xNegative.append(vect.astype(np.float))
                    yNegative.append(0)
            else:
                features = i[5:len(i)-1]
            j+=1
        """yPositive = np.array(yPositive)
        xPositive = np.array(xPositive)
        yNegative = np.array(yNegative)
        xNegative = np.array(xNegative)"""
        features = np.array(features)
        indxP = int(len(xPositive)*.6)
        indxN = int(len(xNegative)*.6)
        xTrain =np.array(xPositive[:indxP] + xNegative[:indxN])
        yTrain = np.array(yPositive[:indxP] + yNegative[:indxN])
        xTest = np.array(xPositive[indxP:] + xNegative[indxN:])
        yTest = np.array(yPositive[indxP:] + yNegative[indxN:])
        
        
            
            
        return xTrain,yTrain,xTest,yTest,features

## Calculate Label Percentage

In [12]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n

## Create Model

In [13]:
def decisionTree():
    return tree.DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

## Evaluation

In [14]:
def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)

def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)

## Cross Validation

In [15]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds, shuffle=True)
    accuracies = []
    i = 1
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        print ("Fold %d Evaluation" %i)
        i +=1
        evaluation(y[test_ind],predictions)
        print ("\n")
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    print ("The average Accuracy is ", avg)

### Read Data

In [16]:
filename = 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-full.csv'
distintCommunities = findCommunities(filename)
XTrain,YTrain,XTest,YTest,Features = extractData(filename, distintCommunities)

### Percentage of Labels

In [17]:
print ("Train Percentage",calcualtePercentage(YTrain))
print ("Test Percentage",calcualtePercentage(YTest))

Train Percentage (0.62761506276150625, 0.3723849372384937)
Test Percentage (0.62703379224030042, 0.37296620775969963)


### Default Decision Tree Model

In [18]:
model= decisionTree()
model.fit(XTrain,YTrain)
predictedY=model.predict(XTest)
evaluation(YTest,predictedY)

Confusion Matrix
[[411  90]
 [111 187]]
Accuracy 0.748435544431
Precision [0.78735632183908044, 0.67509025270758127]
Recall [0.82035928143712578, 0.62751677852348997]


### 10 Fold Cross Validation on Default DecisionTree Model

In [19]:
do_cross_validation(XTrain,YTrain,model,10)

Fold 1 Evaluation
Confusion Matrix
[[50 16]
 [18 36]]
Accuracy 0.716666666667
Precision [0.73529411764705888, 0.69230769230769229]
Recall [0.75757575757575757, 0.66666666666666663]


Fold 2 Evaluation
Confusion Matrix
[[58 16]
 [15 31]]
Accuracy 0.741666666667
Precision [0.79452054794520544, 0.65957446808510634]
Recall [0.78378378378378377, 0.67391304347826086]


Fold 3 Evaluation
Confusion Matrix
[[65 14]
 [14 27]]
Accuracy 0.766666666667
Precision [0.82278481012658233, 0.65853658536585369]
Recall [0.82278481012658233, 0.65853658536585369]


Fold 4 Evaluation
Confusion Matrix
[[52 14]
 [15 39]]
Accuracy 0.758333333333
Precision [0.77611940298507465, 0.73584905660377353]
Recall [0.78787878787878785, 0.72222222222222221]


Fold 5 Evaluation
Confusion Matrix
[[69 15]
 [ 9 27]]
Accuracy 0.8
Precision [0.88461538461538458, 0.6428571428571429]
Recall [0.8214285714285714, 0.75]


Fold 6 Evaluation
Confusion Matrix
[[60 18]
 [17 24]]
Accuracy 0.705882352941
Precision [0.77922077922077926, 0.5

### Top 10 important features 

In [20]:
coef = model.feature_importances_
topindx = np.argsort(coef)[::-1][:10]
print (Features[topindx])


['PctKids2Par' 'racePctWhite' 'racePctHisp' 'PctOccupManu' 'MedYrHousBuilt'
 'MalePctDivorce' 'PctBornSameState' 'PctImmigRec10' 'PctHousLess3BR'
 'numbUrban']
